In [1]:
%matplotlib inline
from utils import *

In [2]:
H5('train')

['/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/14.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/9.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/10.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/8.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/11.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/15.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/7.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/3.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/2.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/6.h5',
 '/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/1.h5',
 '/Users/akamara/Downloads/new-york-cit

In [3]:
def Updator(fpath,readargs=readargs,scaler=scaler):
    import numpy as np
    def degree_to_radion(degree):
        return degree*(np.pi/180)

    def calculate_distance(pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude):

        from_lat = degree_to_radion(pickup_latitude)
        from_long = degree_to_radion(pickup_longitude)
        to_lat = degree_to_radion(dropoff_latitude)
        to_long = degree_to_radion(dropoff_longitude)

        radius = 6371.01

        lat_diff = to_lat - from_lat
        long_diff = to_long - from_long

        a = np.sin(lat_diff / 2)**2 + np.cos(degree_to_radion(from_lat)) * np.cos(degree_to_radion(to_lat)) * np.sin(long_diff / 2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

        return radius * c
    df = pandas.read_hdf(fpath,**readargs)
    df['dist_2'] = calculate_distance(
    df.pickup_latitude,
    df.pickup_longitude,
    df.dropoff_latitude,
    df.dropoff_longitude)
    df = df.query('3<fare_amount < 200 and dist > 0')
    return train_test_split(scaler.transform(df.drop(
    ['pickup_datetime','geometry','fare_amount','key'],axis=1)), df.fare_amount, test_size=0.10, random_state=42)

NameError: name 'scaler' is not defined

In [ ]:
df = pandas.read_hdf('/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/1.h5',**readargs)

In [ ]:
df.describe()

# OUTLIERS

In [ ]:
queries = []
df.fare_amount.plot.box()

In [ ]:
q = '3<fare_amount < 200'

queries.append(q)

df.query(q).shape#fare_amount.plot.box()

In [ ]:
df.shape

In [ ]:
df.query(q).fare_amount.plot.box()

In [ ]:
import numpy as np
def degree_to_radion(degree):
    return degree*(np.pi/180)

def calculate_distance(pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude):
    
    from_lat = degree_to_radion(pickup_latitude)
    from_long = degree_to_radion(pickup_longitude)
    to_lat = degree_to_radion(dropoff_latitude)
    to_long = degree_to_radion(dropoff_longitude)
    
    radius = 6371.01
    
    lat_diff = to_lat - from_lat
    long_diff = to_long - from_long

    a = np.sin(lat_diff / 2)**2 + np.cos(degree_to_radion(from_lat)) * np.cos(degree_to_radion(to_lat)) * np.sin(long_diff / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return radius * c

In [ ]:
df['dist_2'] = calculate_distance(
    df.pickup_latitude,
    df.pickup_longitude,
    df.dropoff_latitude,
    df.dropoff_longitude)

In [ ]:
df.query(' and '.join(queries)).shape#corr().fare_amount.sort_values()

In [ ]:
q2 = 'dist > 0'

queries.append(q2)

df.query(' and '.join(queries)).shape

In [ ]:
df.query(' and '.join(queries)).corr().fare_amount.sort_values()

In [ ]:
df = df.query(' and '.join(queries))

In [ ]:
' and '.join(queries)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler


In [ ]:
scaler = StandardScaler().fit(df.drop(
    ['pickup_datetime','geometry','fare_amount','key'],axis=1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaler.transform(df.drop(
    ['pickup_datetime','geometry','fare_amount','key'],axis=1)), df.fare_amount, test_size=0.10, random_state=42)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [ ]:
X_train.shape,X_eval.shape

In [ ]:
xgbargs = {
    'train_label':y_train.values,
    'train_features_array':X_train,
    'test_label':y_eval.values,
    'test_features_array':X_eval,
    'xgb_params':{
    'objective': 'reg:linear',
    'verbose': True,
    'eta':0.8,
    'xgb_model':None},
    'num_round':100}



In [ ]:
class Train_xgboost :
    '''
    This method aims at easing training for Extreme Gradient boosted trees
    '''
    def __init__(self,train_label,train_features_array,test_features_array,test_label,xgb_params,num_round):
        self.xgb_params = xgb_params
        self.num_round = num_round
        self.train_features_array, self.train_label = train_features_array, train_label
        self.test_features_array, self.test_label   = test_features_array, test_label
    def buil_dmatrix(self):
        self.dtrain = xgb.DMatrix(self.train_features_array, label = self.train_label)
        self.dtest = xgb.DMatrix(self.test_features_array, label = self.test_label)
    def launch_training(self,model):
        evallist = [(self.dtest, 'eval'), (self.dtrain, 'train')]
        plst = self.xgb_params.items()
        self.bst = xgb.train(plst,self.dtrain,self.num_round, evallist,xgb_model=model)
    def learn(self,model=None):
        self.buil_dmatrix()
        self.launch_training(model)

## First step

In [ ]:
XGB                                                    = Train_xgboost(**xgbargs)
XGB.learn()


## second step

In [ ]:
X_train, X_test_2, y_train, y_test_2 = Updator(fpath='/Users/akamara/Downloads/new-york-city-taxi-fare-prediction/ml/src/data/train/2.h5',readargs=readargs,scaler=scaler)

In [ ]:
X_train, X_eval_2, y_train, y_eval_2 = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [ ]:
xgbargs['train_label']               = y_train.values
xgbargs['train_features_array']      = X_train


In [ ]:
XGB2                                                    = Train_xgboost(**xgbargs)
XGB2.learn(XGB.bst)


In [ ]:
predicted.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
pandas.np.sqrt(mean_squared_error(y_test, predicted))

In [ ]:
batch_size = 50
iterations = 25
model = None
for i in range(iterations):
    for start in range(0, len(x_tr), batch_size):
        model = xgb.train({
            'learning_rate': 0.007,
            'update':'refresh',
            'process_type': 'update',
            'refresh_leaf': True,
            #'reg_lambda': 3,  # L2
            'reg_alpha': 3,  # L1
            'silent': False,
        }, dtrain=xgb.DMatrix(x_tr[start:start+batch_size], y_tr[start:start+batch_size]), xgb_model=model)

        y_pr = model.predict(xgb.DMatrix(x_te))
        #print('    MSE itr@{}: {}'.format(int(start/batch_size), sklearn.metrics.mean_squared_error(y_te, y_pr)))
    print('MSE itr@{}: {}'.format(i, sklearn.metrics.mean_squared_error(y_te, y_pr)))

y_pr = model.predict(xgb.DMatrix(x_te))
print('MSE at the end: {}'.format(sklearn.metrics.mean_squared_error(y_te, y_pr))